## Grupo Bimbo Inventory Demand
https://www.kaggle.com/c/grupo-bimbo-inventory-demand

E-commerce website: http://blumonpay.net/bimbo/

In [1]:
dir = "C:/github/kaggle-fiddling/bimbo/"

In [2]:
import numpy as np 
import pandas as pd 
import bcolz

#import ipython_memory_usage as imu
#imu.start_watching_memory()

bcolz.blosc_set_nthreads(8)

8

In [3]:
# read from CSV ; comment-in next lines when done
train = pd.read_csv(dir + 'train.csv', dtype={
    'Semana': np.int8,
    'Agencia_ID': np.int32,
    'Canal_ID': np.int16,
    'Ruta_SAK': np.int16,
    'Cliente_ID': np.int64,
    'Producto_ID': np.int32,
    'Venta_uni_hoy': np.int16,
    'Venta_hoy': np.float16,
    'Dev_uni_proxima': np.int32,
    'Dev_proxima': np.float16,
    'Demanda_uni_equil': np.int16
})

In [4]:
# generate bcolz file; comment-in next lines when done

zTrain = bcolz.ctable.fromdataframe(train)
zTrain.copy(rootdir=dir+"train.bcolz", mode='w')

ctable((74180464,), [('Semana', 'i1'), ('Agencia_ID', '<i4'), ('Canal_ID', '<i2'), ('Ruta_SAK', '<i2'), ('Cliente_ID', '<i8'), ('Producto_ID', '<i4'), ('Venta_uni_hoy', '<i2'), ('Venta_hoy', '<f2'), ('Dev_uni_proxima', '<i4'), ('Dev_proxima', '<f2'), ('Demanda_uni_equil', '<i2')])
  nbytes: 2.28 GB; cbytes: 586.13 MB; ratio: 3.98
  cparams := cparams(clevel=5, shuffle=1, cname='blosclz')
  rootdir := 'C:/github/kaggle-fiddling/bimbo/train.bcolz'
[(3, 1110, 7, 3301, 15766, 1212, 3, 25.140625, 0, 0.0, 3)
 (3, 1110, 7, 3301, 15766, 1216, 4, 33.53125, 0, 0.0, 4)
 (3, 1110, 7, 3301, 15766, 1238, 4, 39.3125, 0, 0.0, 4) ...,
 (9, 25759, 1, 5517, 4528866, 45112, 4, 51.0, 0, 0.0, 4)
 (9, 25759, 1, 5517, 4547943, 40217, 3, 26.9375, 3, 26.9375, 0)
 (9, 25759, 1, 5517, 4708097, 43159, 1, 10.3984375, 0, 0.0, 1)]

In [ ]:
# once bcolz file has been generated, load it in the following way:

trainBZ = bcolz.ctable(rootdir = dir+"train.bcolz")
train = trainBZ.todataframe()

In [5]:
train[['Semana','Demanda_uni_equil']].groupby('Semana').sum()

,Demanda_uni_equil
Semana,
3,77664309.0
4,79618866.0
5,77610637.0
6,73851129.0
7,76597014.0
8,75525105.0
9,75054450.0


<img src="demanda-semana.png" alt="Demand / week" width="240" height="391">

Observations:
- wk 1 = January 1st ?
- no clear trend
- drop in wk 6: holidays ? 
- Mexico is a US-oriented touristic country. demand peek connected to US tourists ? to check this hypothesis, analyze geographical distribution using town_states.csv 